In [1]:
import pandas as pd
import numpy as np
from datetime import date, datetime
import pytz
from matplotlib import pyplot as plt
import csv
from tkinter import *
from tkinter import messagebox, simpledialog, Frame, Label, Entry, Button, Radiobutton, StringVar


class Customer:
    def __init__(self):
        self.transactionhistory = []
        self.fdhistory = []
        self.account_number = None
        self.pin = None
        self.balance = 0
        self.fdbalance = 0
        self.phonenumber = None
        self.fstname = ""
        self.lastname = ""
        self.dob = ""
        self.email = ""
        self.address = ""
        self.age = 0

    def pin_creator(self, pin, confirm_pin):
        try:
            if len(str(pin)) != 6 or not str(pin).isdigit():
                return False, "PIN must be 6 digits"
            if pin != confirm_pin:
                return False, "PINs don't match"
            self.pin = int(pin)
            return True, "PIN set successfully"
        except ValueError:
            return False, "PIN must contain only numbers"

    def account_creation(self, data):
        try:
            self.fstname = data['first_name']
            self.lastname = data['last_name']
            self.dob = data['dob']
            self.dob_date = date.fromisoformat(self.dob)
            today = date.today()
            self.age = today.year - self.dob_date.year - ((today.month, today.day) < (self.dob_date.month, self.dob_date.day))
            
            if len(str(data['phone'])) != 10 or not str(data['phone']).isdigit():
                return False, "Phone number must be 10 digits"
            self.phonenumber = int(data['phone'])
            
            self.balance = 0
            self.fdbalance = 0
            self.email = data['email']
            
            success, msg = self.pin_creator(data['pin'], data['confirm_pin'])
            if not success:
                return False, msg
                
            self.account_number = np.random.randint(1000, 9999)
            self.address = data['address']
            
            with open("userdet.csv", "a", newline="") as f1:
                w1 = csv.writer(f1)
                w1.writerow([self.account_number, self.phonenumber, self.pin, self.balance, 
                           self.age, self.fstname, self.lastname, self.dob, self.email, self.address])
            return True, f"Account created successfully! Account number: {self.account_number}"
        except ValueError:
            return False, "Invalid date format (use YYYY-MM-DD)"
        except Exception as e:
            return False, f"Error: {str(e)}"

    def view_details(self):
        with open("userdet.csv", "r", newline="") as f2:
            r1 = list(csv.reader(f2))
        
        for row in r1:
            if row and int(row[0]) == self.account_number:  # Check if account matches
                return (f"Account Number: {self.account_number}\n"
                        f"Name: {self.fstname} {self.lastname}\n"
                        f"Age: {self.age}\n"
                        f"Phone Number: {self.phonenumber}\n"
                        f"Email: {self.email}\n"
                        f"Address: {self.address}\n"
                        f"Balance: {self.balance}")
        return "Account not found"  # Only return this if no match is found after loop
    def update(self, field, value):
        try:
            with open("userdet.csv", "r", newline="") as f2:
                r1 = list(csv.reader(f2))
            
            found = False
            for row in r1:
                if row and int(row[0]) == self.account_number:
                    found = True
                    if field == "name":
                        names = value.strip().split()
                        if len(names) < 2:
                            return False, "Please provide both first and last names"
                        self.fstname = names[0]
                        self.lastname = " ".join(names[1:])
                        row[5] = self.fstname
                        row[6] = self.lastname
                    elif field == "dob":
                        try:
                            dob_date = date.fromisoformat(value)
                            today = date.today()
                            self.age = today.year - dob_date.year - ((today.month, today.day) < (dob_date.month, dob_date.day))
                            self.dob = value
                            row[7] = self.dob
                            row[4] = str(self.age)
                        except ValueError:
                            return False, "Invalid date format (use YYYY-MM-DD)"
                    elif field == "phone":
                        if len(str(value)) != 10 or not str(value).isdigit():
                            return False, "Phone number must be 10 digits"
                        self.phonenumber = int(value)
                        row[1] = str(self.phonenumber)
                    elif field == "pin":
                        if len(str(value)) != 6 or not str(value).isdigit():
                            return False, "PIN must be 6 digits"
                        self.pin = int(value)
                        row[2] = str(self.pin)
                    elif field == "email":
                        if "@" not in value or "." not in value:  # Basic email validation
                            return False, "Invalid email format"
                        self.email = value
                        row[8] = self.email
                    elif field == "address":
                        if not value.strip():
                            return False, "Address cannot be empty"
                        self.address = value
                        row[9] = self.address
                    break  # Exit loop once the row is found and updated

            if not found:
                return False, "Account not found in database"

            # Write updated data back to CSV
            with open("userdet.csv", "w", newline="") as f1:
                w1 = csv.writer(f1)
                w1.writerows(r1)
            
            return True, f"{field.capitalize()} updated successfully"

        except Exception as e:
            return False, f"Error updating {field}: {str(e)}"

    def self_deposit(self, amount):
        try:
            amount = float(amount)
            if amount <= 0:
                return False, "Amount must be positive"
            
            ist = pytz.timezone('Asia/Kolkata')
            now_ist = datetime.now(ist)
            formatted_now = now_ist.strftime("%Y-%m-%d %H:%M:%S")
            
            with open("userdet.csv", "r", newline="") as f2:
                r1 = list(csv.reader(f2))
            
            for row in r1:
                if row and int(row[0]) == self.account_number:
                    row[3] = float(row[3]) + amount
                    self.balance = float(row[3])
                    break
            
            with open("userdet.csv", "w", newline="") as f1:
                w1 = csv.writer(f1)
                w1.writerows(r1)
            
            with open("t.csv", "a", newline="") as f1:
                w1 = csv.writer(f1)
                w1.writerow([self.account_number, formatted_now, amount, self.balance])
                self.transactionhistory.append((self.account_number, formatted_now, amount, self.balance))
            
            return True, f"Deposit successful. New balance: {self.balance}"
        except Exception as e:
            return False, f"Error: {str(e)}"

    def withdrawal(self, amount, pin):
        try:
            amount = float(amount)
            if amount <= 0:
                return False, "Amount must be positive"
            if int(pin) != self.pin:
                return False, "Incorrect PIN"
            if amount > self.balance:
                return False, "Insufficient balance"
            
            ist = pytz.timezone('Asia/Kolkata')
            now_ist = datetime.now(ist)
            formatted_now = now_ist.strftime("%Y-%m-%d %H:%M:%S")
            
            with open("userdet.csv", "r", newline="") as f2:
                r1 = list(csv.reader(f2))
            
            for row in r1:
                if row and int(row[0]) == self.account_number:
                    row[3] = float(row[3]) - amount
                    self.balance = float(row[3])
                    break
            
            with open("userdet.csv", "w", newline="") as f1:
                w1 = csv.writer(f1)
                w1.writerows(r1)
            
            with open("t.csv", "a", newline="") as f1:
                w1 = csv.writer(f1)
                w1.writerow([self.account_number, formatted_now, -amount, self.balance])
                self.transactionhistory.append((self.account_number, formatted_now, -amount, self.balance))
            
            return True, f"Withdrawal successful. New balance: {self.balance}"
        except Exception as e:
            return False, f"Error: {str(e)}"

    def check_balance(self):
        with open("userdet.csv", "r", newline="") as f2:
            r1 = list(csv.reader(f2))
        for row in r1:
            if row and int(row[0]) == self.account_number:
                return f"Your balance is {float(row[3])}"
        return "Account not found"

    def view_transaction_history(self):
        with open("t.csv", "r", newline="") as f2:
            r1 = list(csv.reader(f2))
        count=0
        hist="     date           ||amount||balnce  \n"
        for row in r1:
            if(int(row[0])==self.account_number):
                count=count+1
                hist=hist+f"{row[1]} || {row[2]} || {row[3]}\n"
                count=count+1
        if count == 0:
            return "No transaction history found"
        else:
            return hist

    def make_fd(self, amount):
        try:
            amount = float(amount)
            if amount > self.balance:
                return False, "Insufficient balance"
            
            ist = pytz.timezone('Asia/Kolkata')
            now_ist = datetime.now(ist)
            formatted_now = now_ist.strftime("%Y-%m-%d %H:%M:%S")
            interest_rate = 7.5 if self.age >= 60 else 6.0
            fd_id = len(self.fdhistory) + 1
            
            with open("fd.csv", "a", newline="") as f1:
                w1 = csv.writer(f1)
                w1.writerow([self.account_number, formatted_now, amount, interest_rate, fd_id, self.pin])
                self.fdhistory.append((formatted_now, amount, interest_rate, fd_id))
            
            success, msg = self.withdrawal(amount, self.pin)
            if not success:
                return False, "FD creation failed: " + msg
                
            return True, f"FD created successfully at {interest_rate}% interest"
        except Exception as e:
            return False, f"Error: {str(e)}"

    def fd_return(self):
        if not self.fdhistory:
            return "No FDs found"
        
        ist = pytz.timezone('Asia/Kolkata')
        now_ist = datetime.now(ist)
        details = "FD_ID\tDate\t\tPrincipal\tInterest\tNet Return\n"
        details += "-"*60 + "\n"
        
        for fd in self.fdhistory:
            fd_date = datetime.strptime(fd[0], "%Y-%m-%d %H:%M:%S")
            fd_date = ist.localize(fd_date)
            years = (now_ist - fd_date).days / 365.25
            principal = float(fd[1])
            interest_rate = float(fd[2])
            net_return = principal * (1 + interest_rate/100) ** years
            details += f"{fd[3]}\t{fd[0]}\t{principal}\t{interest_rate}%\t{net_return:.2f}\n"
        return details

    def fd_withdrawal(self, fd_id, pin):
        try:
            if int(pin) != self.pin:
                return False, "Incorrect PIN"
            
            ist = pytz.timezone('Asia/Kolkata')
            now_ist = datetime.now(ist)
            net_amount = 0
            updated_fd_rows = []
            
            with open("fd.csv", "r", newline="") as f2:
                r1 = list(csv.reader(f2))
            
            found = False
            for row in r1:
                if row and int(row[0]) == self.account_number and int(row[4]) == int(fd_id):
                    found = True
                    fd_date = datetime.strptime(row[1], "%Y-%m-%d %H:%M:%S")
                    fd_date = ist.localize(fd_date)
                    years = (now_ist - fd_date).days / 365.25
                    net_amount = float(row[2]) * (1 + float(row[3])/100) ** years
                else:
                    updated_fd_rows.append(row)
            
            if not found:
                return False, "FD not found"
                
            with open("fd.csv", "w", newline="") as f5:
                w5 = csv.writer(f5)
                w5.writerows(updated_fd_rows)
            
            self.fdhistory = [fd for fd in self.fdhistory if fd[3] != int(fd_id)]
            success, msg = self.self_deposit(net_amount)
            if not success:
                return False, "FD withdrawal failed: " + msg
                
            return True, f"FD withdrawn successfully. Amount credited: {net_amount:.2f}"
        except Exception as e:
            return False, f"Error: {str(e)}"

    def view_spends(self):
        if not self.transactionhistory:
            return False, "No transactions to display"
        
        dates, amounts, balances = [], [], []
        for trans in self.transactionhistory:
            dates.append(trans[1])
            amounts.append(float(trans[2]))
            balances.append(float(trans[3]))
        
        plt.plot(dates, balances)
        plt.xlabel("Date")
        plt.ylabel("Balance")
        plt.title("Spending History")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
        return True, "Graph displayed"


class BankingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("SafeMoney Banking")
        self.root.geometry("800x600")
        self.customer = Customer()
        
        for filename in ["userdet.csv", "t.csv", "fd.csv"]:
            try:
                open(filename, "r").close()
            except FileNotFoundError:
                open(filename, "w").close()

        self.show_main_menu()

    def clear_frame(self):
        for widget in self.root.winfo_children():
            widget.destroy()

    def show_main_menu(self):
        self.clear_frame()
        
        header = Frame(self.root)
        header.pack(pady=20)
        
        title_frame = Frame(header)
        title_frame.pack(side=LEFT)
        Label(title_frame, text="SafeMoney Banking", font=("Arial", 24, "bold")).pack()
        Label(title_frame, text="Secure Banking Solutions", font=("Arial", 12)).pack()
        
        button_frame = Frame(self.root)
        button_frame.pack(pady=50)
        
        Button(button_frame, text="Create Account", width=20, height=2,
               command=self.show_create_account, bg="#4CAF50", fg="white").grid(row=0, column=0, padx=10, pady=10)
        Button(button_frame, text="Login", width=20, height=2,
               command=self.show_login, bg="#2196F3", fg="white").grid(row=0, column=1, padx=10, pady=10)

    def show_create_account(self):
        self.clear_frame()
        
        Label(self.root, text="Create New Account", font=("Arial", 18, "bold")).pack(pady=20)
        
        form_frame = Frame(self.root)
        form_frame.pack(pady=10)
        
        Label(form_frame, text="First Name:").grid(row=0, column=0, sticky="e")
        self.first_name = Entry(form_frame)
        self.first_name.grid(row=0, column=1)
        
        Label(form_frame, text="Last Name:").grid(row=1, column=0, sticky="e")
        self.last_name = Entry(form_frame)
        self.last_name.grid(row=1, column=1)
        
        Label(form_frame, text="DOB (YYYY-MM-DD):").grid(row=2, column=0, sticky="e")
        self.dob = Entry(form_frame)
        self.dob.grid(row=2, column=1)
        
        Label(form_frame, text="Phone Number:").grid(row=3, column=0, sticky="e")
        self.phone = Entry(form_frame)
        self.phone.grid(row=3, column=1)
        
        Label(form_frame, text="Email:").grid(row=4, column=0, sticky="e")
        self.email = Entry(form_frame)
        self.email.grid(row=4, column=1)
        
        Label(form_frame, text="Address:").grid(row=5, column=0, sticky="e")
        self.address = Entry(form_frame)
        self.address.grid(row=5, column=1)
        
        Label(form_frame, text="PIN:").grid(row=6, column=0, sticky="e")
        self.pin = Entry(form_frame, show="*")
        self.pin.grid(row=6, column=1)
        
        Label(form_frame, text="Confirm PIN:").grid(row=7, column=0, sticky="e")
        self.confirm_pin = Entry(form_frame, show="*")
        self.confirm_pin.grid(row=7, column=1)
        
        button_frame = Frame(self.root)
        button_frame.pack(pady=20)
        
        Button(button_frame, text="Create", command=self.create_account,
               bg="#4CAF50", fg="white").pack(side=LEFT, padx=10)
        Button(button_frame, text="Back", command=self.show_main_menu,
               bg="#f44336", fg="white").pack(side=LEFT, padx=10)

    def create_account(self):
        data = {
            'first_name': self.first_name.get(),
            'last_name': self.last_name.get(),
            'dob': self.dob.get(),
            'phone': self.phone.get(),
            'email': self.email.get(),
            'address': self.address.get(),
            'pin': self.pin.get(),
            'confirm_pin': self.confirm_pin.get()
        }
        
        for key, value in data.items():
            if not value:
                messagebox.showerror("Error", f"Please fill all fields")
                return
                
        success, msg = self.customer.account_creation(data)
        if success:
            messagebox.showinfo("Success", msg)
            self.show_logged_in_menu()
        else:
            messagebox.showerror("Error", msg)

    def show_login(self):
        self.clear_frame()
        
        Label(self.root, text="Login", font=("Arial", 18, "bold")).pack(pady=20)
        
        form_frame = Frame(self.root)
        form_frame.pack(pady=10)
        
        Label(form_frame, text="Login with:").grid(row=0, column=0, sticky="e")
        self.login_type = StringVar(value="account")
        Radiobutton(form_frame, text="Account Number", variable=self.login_type,
                   value="account").grid(row=0, column=1)
        Radiobutton(form_frame, text="Phone Number", variable=self.login_type,
                   value="phone").grid(row=0, column=2)
        
        Label(form_frame, text="ID:").grid(row=1, column=0, sticky="e")
        self.login_id = Entry(form_frame)
        self.login_id.grid(row=1, column=1, columnspan=2)

        Label(form_frame, text="PIN:").grid(row=2, column=0, sticky="e")
        self.pin_1 = Entry(form_frame, show="*")
        self.pin_1.grid(row=2, column=1, columnspan=2)
        
        button_frame = Frame(self.root)
        button_frame.pack(pady=20)
        
        Button(button_frame, text="Login", command=self.login,
               bg="#4CAF50", fg="white").pack(side=LEFT, padx=10)
        Button(button_frame, text="Back", command=self.show_main_menu,
               bg="#f44336", fg="white").pack(side=LEFT, padx=10)

    def login(self):
        login_id = self.login_id.get().strip()
        pin_1 = self.pin_1.get().strip()

        if not login_id:
            messagebox.showerror("Error", "Please enter login ID")
            return
        if not pin_1:
            messagebox.showerror("Error", "Please enter PIN")
            return

        print(f"Entered PIN: {pin_1}")  # Debugging

        try:
            with open("userdet.csv", "r") as f2:
                r1 = csv.reader(f2)
                found = False
                for row in r1:
                    if not row:  # Skip empty rows
                        continue
                    print(f"CSV Row: {row}")  # Debugging
                    print(f"CSV PIN: {row[2]}")  # Debugging
                    if ((self.login_type.get() == "account" and row[0] == login_id) or 
                        (self.login_type.get() == "phone" and row[1] == login_id)) and row[2] == pin_1:
                        # Load basic customer details
                        self.customer.account_number = int(row[0])
                        self.customer.phonenumber = int(row[1])
                        self.customer.pin = int(row[2])  # Store as int
                        self.customer.balance = float(row[3])
                        self.customer.age = int(row[4])
                        self.customer.fstname = row[5]
                        self.customer.lastname = row[6]
                        self.customer.dob = row[7]
                        self.customer.email = row[8]
                        self.customer.address = row[9]
                        found = True
                        break
                
                if not found:
                    messagebox.showerror("Error", "Invalid ID or PIN")
                    return

            # Load transaction history from t.csv
            self.customer.transactionhistory = []
            try:
                with open("t.csv", "r", newline="") as f3:
                    r2 = csv.reader(f3)
                    for row in r2:
                        if row and int(row[0]) == self.customer.account_number:
                            self.customer.transactionhistory.append((int(row[0]), row[1], float(row[2]), float(row[3])))
            except FileNotFoundError:
                pass  # If t.csv doesn't exist yet, skip

            # Load FD history from fd.csv
            self.customer.fdhistory = []
            try:
                with open("fd.csv", "r", newline="") as f4:
                    r3 = csv.reader(f4)
                    for row in r3:
                        if row and int(row[0]) == self.customer.account_number:
                            self.customer.fdhistory.append((row[1], float(row[2]), float(row[3]), int(row[4])))
            except FileNotFoundError:
                pass  # If fd.csv doesn't exist yet, skip

            if found:
                messagebox.showinfo("Success", "Login successful")
                self.show_logged_in_menu()
        except FileNotFoundError:
            messagebox.showerror("Error", "User data file not found")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

    def show_logged_in_menu(self):
        self.clear_frame()
        
        Label(self.root, text=f"Welcome {self.customer.fstname}", font=("Arial", 18, "bold")).pack(pady=20)
        
        button_frame = Frame(self.root)
        button_frame.pack(pady=20)
        
        buttons = [
            ("View Details", self.show_details, "#4CAF50"),
            ("Update Details", self.show_update_menu, "#2196F3"),
            ("Deposit", self.show_deposit, "#FF9800"),
            ("Withdraw", self.show_withdraw, "#9C27B0"),
            ("Check Balance", self.show_balance, "#607D8B"),
            ("Transaction History", self.show_transaction_history, "#4CAF50"),
            ("Make FD", self.show_make_fd, "#2196F3"),
            ("View FDs", self.show_fd_return, "#FF9800"),
            ("Withdraw FD", self.show_fd_withdraw, "#9C27B0"),
            ("View Spends", self.show_spends, "#607D8B"),
            ("Logout", self.show_main_menu, "#f44336")
        ]
        
        for i, (text, command, color) in enumerate(buttons):
            Button(button_frame, text=text, width=20, height=2,
                   command=command, bg=color, fg="white").grid(row=i//2, column=i%2, padx=10, pady=10)

    def show_details(self):
        self.clear_frame()
        Label(self.root, text="Account Details", font=("Arial", 18, "bold")).pack(pady=20)
        
        text = Text(self.root, height=10, width=60)
        text.insert(END, self.customer.view_details())
        text.config(state=DISABLED)
        text.pack(pady=10)
        
        Button(self.root, text="Back", command=self.show_logged_in_menu,
               bg="#f44336", fg="white").pack(pady=20)

    def show_update_menu(self):
        self.clear_frame()
        Label(self.root, text="Update Details", font=("Arial", 18, "bold")).pack(pady=20)
        
        button_frame = Frame(self.root)
        button_frame.pack(pady=20)
        
        updates = [("Name", "name"), ("DOB", "dob"), ("Phone", "phone"),
                  ("PIN", "pin"), ("Email", "email"), ("Address", "address")]
        colors = ["#4CAF50", "#2196F3", "#FF9800", "#9C27B0", "#607D8B", "#00BCD4"]
        
        for i, (text, field) in enumerate(updates):
            Button(button_frame, text=text, width=20, command=lambda f=field : self.update_field(f),
                   bg=colors[i], fg="white").grid(row=i//2, column=i%2, padx=10, pady=10)
        
        Button(button_frame, text="Back", width=20, command=self.show_logged_in_menu,
               bg="#f44336", fg="white").grid(row=3, column=1, padx=10, pady=10)

    def update_field(self, field):
        if field == "name":
            value = simpledialog.askstring("Update", "Enter full name (First Last):")
        elif field == "pin":
            value = simpledialog.askstring("Update", "Enter new 6-digit PIN:", show="*")
        else:
            value = simpledialog.askstring("Update", f"Enter new {field}:")
        
        if value is not None:  # Check if user didn't cancel
            success, msg = self.customer.update(field, value)
            if success:
                messagebox.showinfo("Success", msg)
            else:
                messagebox.showerror("Error", msg)
        else:
            messagebox.showinfo("Info", "Update cancelled")

    def show_deposit(self):
        amount = simpledialog.askfloat("Deposit", "Enter amount:")
        if amount:
            success, msg = self.customer.self_deposit(amount)
            messagebox.showinfo("Result", msg) if success else messagebox.showerror("Error", msg)

    def show_withdraw(self):
        amount = simpledialog.askfloat("Withdraw", "Enter amount:")
        if amount:
            pin = simpledialog.askstring("PIN", "Enter PIN:", show="*")
            if pin:
                success, msg = self.customer.withdrawal(amount, pin)
                messagebox.showinfo("Result", msg) if success else messagebox.showerror("Error", msg)

    def show_balance(self):
        messagebox.showinfo("Balance", self.customer.check_balance())

    def show_transaction_history(self):
        self.clear_frame()
        Label(self.root, text="Transaction History", font=("Arial", 18, "bold")).pack(pady=20)
        
        text = Text(self.root, height=15, width=60)
        text.insert(END, self.customer.view_transaction_history())
        text.config(state=DISABLED)
        text.pack(pady=10)
        
        Button(self.root, text="Back", command=self.show_logged_in_menu,
               bg="#f44336", fg="white").pack(pady=20)

    def show_make_fd(self):
        amount = simpledialog.askfloat("Fixed Deposit", "Enter amount:")
        if amount:
            success, msg = self.customer.make_fd(amount)
            messagebox.showinfo("Result", msg) if success else messagebox.showerror("Error", msg)

    def show_fd_return(self):
        self.clear_frame()
        Label(self.root, text="Fixed Deposits", font=("Arial", 18, "bold")).pack(pady=20)
        
        text = Text(self.root, height=15, width=60)
        text.insert(END, self.customer.fd_return())
        text.config(state=DISABLED)
        text.pack(pady=10)
        
        Button(self.root, text="Back", command=self.show_logged_in_menu,
               bg="#f44336", fg="white").pack(pady=20)

    def show_fd_withdraw(self):
        fd_id = simpledialog.askinteger("Withdraw FD", "Enter FD ID:")
        if fd_id:
            pin = simpledialog.askstring("PIN", "Enter PIN:", show="*")
            if pin:
                success, msg = self.customer.fd_withdrawal(fd_id, pin)
                messagebox.showinfo("Result", msg) if success else messagebox.showerror("Error", msg)

    def show_spends(self):
        success, msg = self.customer.view_spends()
        if not success:
            messagebox.showinfo("Info", msg)


if __name__ == "__main__":
    root = Tk()
    app = BankingApp(root)
    root.mainloop()